In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from xgboost import XGBClassifier
import seaborn as sns
sns.set_style('white')

In [40]:
features = ['loan_amnt', 'annual_inc', 'term','funded_amnt','int_rate','installment','dti','delinq_2yrs','revol_bal','total_acc','total_pymnt']

In [41]:
a = pd.read_csv('accepted_2007_to_2018Q4.csv', usecols=features+['loan_status'])
a

,loan_amnt,funded_amnt,term,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,revol_bal,total_acc,total_pymnt
0,3600.0,3600.0,36 months,13.99,123.03,55000.0,Fully Paid,5.91,0.0,2765.0,13.0,4421.723917
1,24700.0,24700.0,36 months,11.99,820.28,65000.0,Fully Paid,16.06,1.0,21470.0,38.0,25679.660000
2,20000.0,20000.0,60 months,10.78,432.66,63000.0,Fully Paid,10.78,0.0,7869.0,18.0,22705.924294
3,35000.0,35000.0,60 months,14.85,829.90,110000.0,Current,17.06,0.0,7802.0,17.0,31464.010000
4,10400.0,10400.0,60 months,22.45,289.91,104433.0,Fully Paid,25.37,1.0,21929.0,35.0,11740.500000
...,...,...,...,...,...,...,...,...,...,...,...,...
2260696,40000.0,40000.0,60 months,10.49,859.56,227000.0,Current,12.75,7.0,8633.0,37.0,24903.930000
2260697,24000.0,24000.0,60 months,14.49,564.56,110000.0,Charged Off,18.30,0.0,17641.0,31.0,6755.400000
2260698,14000.0,14000.0,60 months,14.49,329.33,95000.0,Current,23.36,0.0,7662.0,22.0,9621.250000
2260699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
full_set = a.query("loan_status in ('Fully Paid', 'Charged Off')").copy()

In [43]:
full_set['loan_status'] = full_set['loan_status'].replace(
    {
        'Fully Paid': 1,
        'Charged Off': 0
    }
)

full_set['term'] = full_set['term'].replace(
    {
        ' 36 months': 36,
        ' 60 months': 60
    }
)

In [44]:
MinMaxScaler(full_set)

MinMaxScaler(feature_range=         loan_amnt  funded_amnt  term  int_rate  installment  annual_inc  \
0           3600.0       3600.0    36     13.99       123.03     55000.0   
1          24700.0      24700.0    36     11.99       820.28     65000.0   
2          20000.0      20000.0    60     10.78       432.66     63000.0   
4          10400.0      10400.0    60     22.45       289.91    104433.0   
5          11950.0      11950.0    36     13.44       405.18     34000.0   
...            ...          ...   ...       ...          ...         ...   
2260688    18000.0      18000.0    60      9.49       377.95    130000.0   
2260690    29400.0      29400.0    60     13.99       683.9...
1                  1  16.06          1.0    21470.0       38.0  25679.660000  
2                  1  10.78          0.0     7869.0       18.0  22705.924294  
4                  1  25.37          1.0    21929.0       35.0  11740.500000  
5                  1  10.20          0.0     8822.0        6.0  1

In [45]:
model_features = list(full_set.columns)
model_features.remove('loan_status')

In [46]:
full_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1345310 entries, 0 to 2260697
Data columns (total 12 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   loan_amnt    1345310 non-null  float64
 1   funded_amnt  1345310 non-null  float64
 2   term         1345310 non-null  int64  
 3   int_rate     1345310 non-null  float64
 4   installment  1345310 non-null  float64
 5   annual_inc   1345310 non-null  float64
 6   loan_status  1345310 non-null  int64  
 7   dti          1344936 non-null  float64
 8   delinq_2yrs  1345310 non-null  float64
 9   revol_bal    1345310 non-null  float64
 10  total_acc    1345310 non-null  float64
 11  total_pymnt  1345310 non-null  float64
dtypes: float64(10), int64(2)
memory usage: 133.4 MB


In [55]:
X = full_set[model_features]
y = full_set['loan_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)


# model accuracy
result = []

XGB = XGBClassifier(max_depth = 1)
XGB.fit(X_train, y_train)
y_pred_xgb = XGB.predict(X_test)
cm_xgb = confusion_matrix(y_pred_xgb, y_test)
acc_xgb = accuracy_score(y_test, y_pred_xgb)
result.append(acc_xgb)

results = pd.DataFrame({"Model Result": result,
                        "Models": ["XGBoost"]})
results


/Users/zhukate/opt/anaconda3/envs/py39/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/zhukate/opt/anaconda3/envs/py39/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[07:35:23] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,Model Result,Models
0,0.933183,XGBoost
